# Seq2Seq
### Neural Machine Translation using word level language model and embeddings in Keras

- https://medium.com/@dev.elect.iitd/neural-machine-translation-using-word-level-seq2seq-model-47538cba8cd7
- https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py
- https://machinelearningmastery.com/define-encoder-decoder-sequence-sequence-model-neural-machine-translation-keras/
- https://machinelearningmastery.com/develop-encoder-decoder-model-sequence-sequence-prediction-keras/

---

In [1]:
import pandas as pd
import numpy as np

import string
from itertools import chain
import pickle
import re

from string import digits

import matplotlib.pyplot as plt

# Load Data

In [107]:
# load
with open('df_story_desc_final.pickle', 'rb') as f:
    df_story_and_desc = pickle.load(f)

print ("before: ", df_story_and_desc.shape)
df_story_and_desc = df_story_and_desc[:30000]

df_story_and_desc_id = df_story_and_desc['Story_Photo_id']
df_story_and_desc_text = df_story_and_desc[['Desc', 'Story']]

print ("after: ", df_story_and_desc_text.shape)
df_story_and_desc_text.head(15)

before:  (145116, 4)
after:  (30000, 2)


,Desc,Story
0,big old tree being photographed on a sunny day,"and its magnificent trunk , larger than life i..."
1,a old curvy tree in the sun light .,"and its magnificent trunk , larger than life i..."
2,a person is taking a picture of a large tree a...,"and its magnificent trunk , larger than life i..."
3,large tree with many outstretching branches an...,we found this tree when we were walking in a n...
4,a green sign is describing a historic tree and...,it turns out it is a popular attraction here .
5,a large tree with roots that look like crocodi...,"the tree is very unusual , with its roots expo..."
6,big old tree being photographed on a sunny day,"the trunk was really wide , as much as 12 feet !"
7,huge brown tree roots rose above the ground .,you can see how big these roots are - pretty a...
8,a large tree with many branches coming out,we found this tree when we were walking in a n...
9,a plaque describes an historical tree and advi...,it turns out it is a popular attraction here .


==> 15개 단위로 같은 사진 set에 대한 내용

In [108]:
df_story_and_desc_text[df_story_and_desc_text.Desc=='big old tree being photographed on a sunny day']

,Desc,Story
0,big old tree being photographed on a sunny day,"and its magnificent trunk , larger than life i..."
6,big old tree being photographed on a sunny day,"the trunk was really wide , as much as 12 feet !"
18,big old tree being photographed on a sunny day,some more different parts of the tree .
24,big old tree being photographed on a sunny day,the trunk was incredibly thick and rigid .
39,big old tree being photographed on a sunny day,i was dwarfed by the tree 's size .


In [109]:
for i,row in df_story_and_desc_text[df_story_and_desc_text.Desc=='big old tree being photographed on a sunny day'].iterrows():
    print("pair",i,":",row['Desc']+" ==> "+row['Story'])

pair 0 : big old tree being photographed on a sunny day ==> and its magnificent trunk , larger than life itself .
pair 6 : big old tree being photographed on a sunny day ==> the trunk was really wide , as much as 12 feet !
pair 18 : big old tree being photographed on a sunny day ==> some more different parts of the tree .
pair 24 : big old tree being photographed on a sunny day ==> the trunk was incredibly thick and rigid .
pair 39 : big old tree being photographed on a sunny day ==> i was dwarfed by the tree 's size .


==> 하나의 Description Text에 5개의 Story가 대응됨

## cleaning

In [110]:
def re_sub(item):
    re_sentence = []
    for sentence in item:
        sentence = re.sub('[^a-z0-9A-Z]+', ' ', sentence)
        re_sentence.append(sentence)
    return re_sentence

In [111]:
clean_data = df_story_and_desc_text.apply(lambda x: re_sub(x))
clean_data.columns = ['in_desc','out_story']
clean_data['out_story'] = clean_data['out_story'].apply(lambda x : '<sos> '+ x + ' <eos>')
clean_data

,in_desc,out_story
0,big old tree being photographed on a sunny day,<sos> and its magnificent trunk larger than li...
1,a old curvy tree in the sun light,<sos> and its magnificent trunk larger than li...
2,a person is taking a picture of a large tree a...,<sos> and its magnificent trunk larger than li...
3,large tree with many outstretching branches an...,<sos> we found this tree when we were walking ...
4,a green sign is describing a historic tree and...,<sos> it turns out it is a popular attraction ...
...,...,...
29995,a crowd of people in a village square three of...,<sos> we went to organization last summer for ...
29996,people on a safari truck watching as they expl...,<sos> we got to ride so many different rides ...
29997,children pose for a photograph on steps in a p...,<sos> the kids were having so much fun <eos>
29998,kids wearing pirate hats are brandishing toy s...,<sos> they loved going around to all the diffe...


In [ ]:
clean_data = clean_data.

# Vectorize the data

--- 

0907 DONE

---

In [73]:
desc_words = set()
for desc_paragraph in clean_data.in_desc:
    for word in desc_paragraph.split():
        if word not in desc_words:
            desc_words.add(word)
            
story_words = set()
for story_parapgraph in clean_data.out_story:
    for word in story_parapgraph.split():
        if word not in story_words:
            story_words.add(word)
    
len(desc_words), len(story_words)

(2865, 3486)

In [74]:
input_words = sorted(list(desc_words))
target_words = sorted(list(story_words))

num_encoder_tokens = len(desc_words)
num_decoder_tokens = len(story_words)

max_encoder_seq_length = max([len(txt.split(" ")) for txt in clean_data['in_desc']])
max_decoder_seq_length = max([len(txt.split(" ")) for txt in clean_data['out_story']])

print('Number of samples:', len(input_words))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 2865
Number of unique input tokens: 2865
Number of unique output tokens: 3486
Max sequence length for inputs: 45
Max sequence length for outputs: 47


In [75]:
len(clean_data.out_story), max_decoder_seq_length, num_decoder_tokens

(10000, 47, 3486)

In [76]:
encoder_input_data = np.zeros(
    (len(clean_data.in_desc), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(clean_data.out_story), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(clean_data.out_story), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

메모리 뻑남..

MemoryError: Unable to allocate 59.8 GiB for an array with shape (30000, 82, 6524) and data type float32


In [77]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(10000, 45)
(10000, 47)
(10000, 47, 3486)


**why decoder_target_data.shape is 3d**

- 모든 단어에 대하여 이전 단어로부터 다음 단어를 예측하는 소프트맥스 층을 가지기 때문에 

--- 

0908 DONE

- 왜 1로 초기화 하는가

    - ==> 초기화를 1로 하는 것이 아니라 t=0을 건너뛰는 것임

---

In [78]:
input_token_index = dict([(word, i) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i) for i, word in enumerate(target_words)])

In [79]:
for i, (input_text, target_text) in enumerate(zip(clean_data['in_desc'], clean_data['out_story'])):

    # encoder
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]

    # decoder
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]  
        if t > 0: 
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [80]:
encoder_input_data.shape

(10000, 45)

In [81]:
print (decoder_input_data.shape)
decoder_input_data

(10000, 47)


array([[  27.,  125., 1676., ...,    0.,    0.,    0.],
       [  27.,  125., 1676., ...,    0.,    0.,    0.],
       [  27.,  125., 1676., ...,    0.,    0.,    0.],
       ...,
       [  27., 1674., 3347., ...,    0.,    0.,    0.],
       [  27., 3368.,   96., ...,    0.,    0.,    0.],
       [  27., 3368., 1096., ...,    0.,    0.,    0.]], dtype=float32)

In [82]:
print(decoder_target_data.shape)
decoder_target_data

(10000, 47, 3486)


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

--- 

0909 DONE

---

# Build keras encoder-decoder model

http://incredible.ai/nlp/2020/02/20/Sequence-To-Sequence-with-Attention/
https://docs.chainer.org/en/stable/examples/seq2seq.html

https://tykimos.github.io/2018/09/14/ten-minute_introduction_to_sequence-to-sequence_learning_in_Keras/

In [83]:
import tensorflow as tf
from tensorflow import keras

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Bidirectional

In [93]:
batch_size = 64  # Batch size for training.
epochs = 50  # Number of epochs to train for.
latent_dim = 64  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
embedding_size = 50 # Embedding Dimension

In [94]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
print('encoder_inputs: ', encoder_inputs.shape)

# English words embedding
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
print('en_x: ', en_x.shape)

encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
print('encoder_outputs: ', encoder_outputs.shape)
print('state_h: ', state_h.shape)
print('state_c: ', state_c.shape)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

encoder_inputs:  (None, None)
en_x:  (None, None, 50)
encoder_outputs:  (None, 64)
state_h:  (None, 64)
state_c:  (None, 64)


In [95]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, ))
print('decoder_inputs: ', decoder_inputs.shape)

# french word embeddings
dex=  Embedding(num_decoder_tokens, embedding_size)
final_dex = dex(decoder_inputs)

# We set up our decoder to return full output sequences, and to return internal states as well. 
# We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
print('decoder_outputs: ', decoder_outputs.shape)

decoder_inputs:  (None, None)
decoder_outputs:  (None, None, 3486)


--- 

0911 DONE

---

In [96]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_28 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, None, 50)     143250      input_27[0][0]                   
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, None, 50)     174300      input_28[0][0]                   
______________________________________________________________________________________

In [97]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(10000, 45)
(10000, 47)
(10000, 47, 3486)


In [98]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [99]:
with tf.device('/gpu:0'):
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2)

Epoch 1/50
125/125 [==============================] - 39s 313ms/step - loss: 1.4385 - accuracy: 0.0229 - val_loss: 1.5800 - val_accuracy: 0.0279
Epoch 2/50
125/125 [==============================] - 38s 306ms/step - loss: 1.3246 - accuracy: 0.0294 - val_loss: 1.5696 - val_accuracy: 0.0294
Epoch 3/50
125/125 [==============================] - 38s 307ms/step - loss: 1.2922 - accuracy: 0.0302 - val_loss: 1.5640 - val_accuracy: 0.0296
Epoch 4/50
125/125 [==============================] - 38s 304ms/step - loss: 1.2634 - accuracy: 0.0318 - val_loss: 1.5526 - val_accuracy: 0.0321
Epoch 5/50
125/125 [==============================] - 39s 311ms/step - loss: 1.2330 - accuracy: 0.0354 - val_loss: 1.5325 - val_accuracy: 0.0353
Epoch 6/50
125/125 [==============================] - 38s 306ms/step - loss: 1.2008 - accuracy: 0.0387 - val_loss: 1.5202 - val_accuracy: 0.0374
Epoch 7/50
125/125 [==============================] - 39s 310ms/step - loss: 1.1696 - accuracy: 0.0419 - val_loss: 1.4983 - val_ac

결과가.. 이상..

In [43]:
# Save model
model.save('s2s.h5')

In [106]:
model = keras.models.load_model('s2s.h5')

# Sentence Generation

In [100]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_8 (Embedding)      (None, None, 50)          143250    
_________________________________________________________________
lstm_12 (LSTM)               [(None, 64), (None, 64),  29440     
Total params: 172,690
Trainable params: 172,690
Non-trainable params: 0
_________________________________________________________________


In [101]:
# Redefine the decoder model with decoder will be getting below inputs from encoder while in prediction
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)
print(final_dex2)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

# sampling model will take encoder states and decoder_input(seed initially) and output the predictions(french word index) We dont care about decoder_states2
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

Tensor("embedding_9/embedding_lookup_1/Identity_1:0", shape=(None, None, 50), dtype=float32)


In [102]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [103]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['<sos>']
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
       
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        # Exit condition: either hit max length or find stop character.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 52):
            stop_condition = True
        
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        
        # Update states
        states_value = [h, c]
    return decoded_sentence

In [104]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', clean_data['in_desc'][seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: big old tree being photographed on a sunny day
Decoded sentence:  the kids were very excited to the crowd of the porch
-
Input sentence: a old curvy tree in the sun light 
Decoded sentence:  the kids were very excited to the crowd of the porch
-
Input sentence: a person is taking a picture of a large tree and you can see their shadow
Decoded sentence:  the kids were very excited to the crowd of the porch
-
Input sentence: large tree with many outstretching branches and leaves 
Decoded sentence:  the kids were very excited to the crowd of the porch
-
Input sentence: a green sign is describing a historic tree and the rules for visiting it 
Decoded sentence:  the kids were very excited to the crowd of the porch
-
Input sentence: a large tree with roots that look like crocodile tails 
Decoded sentence:  the kids were very excited to the crowd of the porch
-
Input sentence: big old tree being photographed on a sunny day
Decoded sentence:  the kids were very excited to the 

---
- epoch 를 줄여야 어느정도 학습 가능
- input에 대응하는 output pair 가 너무 많아서 성능이 안좋은게아닐까? => input : output 1대1 대응만 남기기